In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from fastai import *
from fastai.vision import *
from fastai.callbacks import *

In [3]:
folder = Path('D:/ProgProjekte/Python/Results-Exact-Study/Patches')

In [4]:
path = folder/"Annotation/NrExperts"

In [5]:
tfms = get_transforms(do_flip=True, 
                      flip_vert=True, 
                      max_rotate=90,  
                      #max_lighting=0.0, 
                      #max_zoom=2, 
                      #max_warp=0.2,
                      #p_affine=0.75,
                      #p_lighting=0.75,  
                      #xtra_tfms=xtra_tfms,
                     )

In [6]:
def get_data(folder, bs,size):
    data = ImageDataBunch.from_folder(folder, train="train", valid="val", size=size, bs=bs, ds_tfms=tfms, num_workers=0)
    return data.normalize()

In [7]:
results = {}
for user in path.glob("*"):
    folder = path/user
    
    acc_values = []
    for i in range(5):
        learn = cnn_learner(get_data(folder, bs=64, size=64), models.resnet18, metrics=[accuracy], callback_fns=[])

        lr = 0.05
        learn.fit_one_cycle(3, slice(lr))
        learn.unfreeze()

        lr = 0.001
        learn.fit_one_cycle(5, slice(lr))
        
        acc_values.append(float(np.max(learn.recorder.metrics)))  
    
    results[user.stem] = np.mean(acc_values)

epoch,train_loss,valid_loss,accuracy,time
0,0.868522,1.084979,0.643982,00:28
1,0.713825,0.583513,0.717098,00:29
2,0.566810,0.480556,0.787964,00:29


epoch,train_loss,valid_loss,accuracy,time
0,0.425216,0.405331,0.844207,00:30
1,0.396764,0.384583,0.831271,00:30
2,0.368875,0.380327,0.839708,00:31
3,0.337959,0.339028,0.858268,00:31
4,0.325875,0.342691,0.851519,00:31


epoch,train_loss,valid_loss,accuracy,time
0,0.858050,1.131396,0.607987,00:29
1,0.672703,0.620085,0.690664,00:31
2,0.527828,0.472003,0.785714,00:33


epoch,train_loss,valid_loss,accuracy,time
0,0.445101,0.426719,0.814961,00:32
1,0.390960,0.419217,0.832958,00:32
2,0.366948,0.363407,0.840832,00:32
3,0.346298,0.330353,0.858830,00:31
4,0.324175,0.331844,0.859393,00:31


epoch,train_loss,valid_loss,accuracy,time
0,0.827568,0.872424,0.709224,00:30
1,0.699197,0.411738,0.815523,00:30
2,0.542083,0.459323,0.786839,00:31


...

epoch,train_loss,valid_loss,accuracy,time
0,0.408184,0.413748,0.842520,00:18
1,0.381287,0.353305,0.856018,00:17
2,0.339771,0.377032,0.858268,00:17
3,0.327908,0.353875,0.851519,00:17
4,0.309874,0.349056,0.859393,00:17


epoch,train_loss,valid_loss,accuracy,time
0,0.793911,0.932584,0.699663,00:17
1,0.637862,0.465253,0.790776,00:17
2,0.532581,0.525397,0.740720,00:17


epoch,train_loss,valid_loss,accuracy,time
0,0.424639,0.417606,0.819460,00:18
1,0.376033,0.338455,0.853206,00:17
2,0.352244,0.328667,0.867829,00:17
3,0.327779,0.328845,0.868954,00:17
4,0.312410,0.331006,0.865579,00:18


epoch,train_loss,valid_loss,accuracy,time
0,0.810930,1.822333,0.578740,00:17
1,0.672850,0.509344,0.767154,00:17
2,0.526800,0.479180,0.786277,00:17


epoch,train_loss,valid_loss,accuracy,time
0,0.418943,0.420447,0.827334,00:17
1,0.392359,0.342542,0.854893,00:18
2,0.357752,0.367265,0.851519,00:17
3,0.326992,0.346974,0.857705,00:17
4,0.312180,0.336609,0.861080,00:17


In [8]:
results

{'NrExperts_10': 0.8598425269126893,
 'NrExperts_2': 0.8482564806938171,
 'NrExperts_3': 0.8714285731315613,
 'NrExperts_4': 0.8906636595726013,
 'NrExperts_5': 0.883689534664154,
 'NrExperts_6': 0.8813273310661316,
 'NrExperts_7': 0.8737907767295837,
 'NrExperts_8': 0.871091115474701,
 'NrExperts_9': 0.8610798716545105}

In [9]:
import json
with open('NrExperts_Annotation.json', 'w') as fp:
    json.dump(results, fp)